<a href="https://colab.research.google.com/github/Ahnkyuwon504/AI-modeling/blob/main/eval-rag-and-instruct/gemma_ko_2b_rag_pymongo_med.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 라이브러리 설치

In [1]:
!pip install -q datasets pandas pymongo sentence_transformers
!pip install -q -U transformers
!pip install -q accelerate
!pip install -q "pymongo[srv]"
!pip install -q -U bitsandbytes

# 2. Dataset

## 2.1. Preprocessing

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
from datasets import load_dataset
import pandas as pd

# https://huggingface.co/datasets/mncai/kin_med_2M?row=12
dataset = load_dataset("mncai/kin_med_2M", split="train")

In [4]:
print(dataset.shape)
small_dataset = dataset.select(range(int(len(dataset) * 0.001)))

(2019893, 3)


In [5]:
print(small_dataset.shape)
print(small_dataset.select(range(3)))

(2019, 3)
Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 3
})


In [6]:
# dataframe으로 변환
dataset_df = pd.DataFrame(small_dataset)

dataset_df.head(5)

,instruction,input,output
0,벌어진 앞니 레진 떼우기 이정도 벌어진 앞니 음식 씹을 때마다 너무 불편해레진 떼우...,,가능은 하지만 일단 색이 치아 색과 맞지 않습니다 그리고 잘 떨어진다는 단점이 있습...
1,"유전형 탈모 검사법이 있는지 궁금합니다. 저희집안은 친가, 외가를 통틀어서 탈모가 ...",,유전성 남성형 탈모에 있어서 탈모유전자가 하나는아니며유전자 검사가 있지만 그검사는...
2,고추에 때? 흰색 물질이 있어요(급) 고추를 까보면 귀두에 흰색이 있는데요 일반 오...,,음경부분에 흰색물질이 발견되어 고민이시군요.육안으로 확인되지 않아 판단이 어려우나 ...
3,17년12월 유방섬유선종 1.15cm 유방외과 선생님 좀 봐주세요.17년12월 유방...,,일단 성장하는 혹은 제거하는 것이 좋습니다. 그런데 약 2년 동안 6.5mm 커진 ...
4,처녀막 재생수술 궁금증 안녕하세요. 제가 요즘 고민이 하나 생겼는데요. 아직 성...,,"처녀막의 검사는 질입구에서 확인이 되므로 특별한 기구를 사용치 않으며, 간단합니다..."


In [7]:
dataset_df = dataset_df.drop(columns=["input"])
dataset_df.head(5)

,instruction,output
0,벌어진 앞니 레진 떼우기 이정도 벌어진 앞니 음식 씹을 때마다 너무 불편해레진 떼우...,가능은 하지만 일단 색이 치아 색과 맞지 않습니다 그리고 잘 떨어진다는 단점이 있습...
1,"유전형 탈모 검사법이 있는지 궁금합니다. 저희집안은 친가, 외가를 통틀어서 탈모가 ...",유전성 남성형 탈모에 있어서 탈모유전자가 하나는아니며유전자 검사가 있지만 그검사는...
2,고추에 때? 흰색 물질이 있어요(급) 고추를 까보면 귀두에 흰색이 있는데요 일반 오...,음경부분에 흰색물질이 발견되어 고민이시군요.육안으로 확인되지 않아 판단이 어려우나 ...
3,17년12월 유방섬유선종 1.15cm 유방외과 선생님 좀 봐주세요.17년12월 유방...,일단 성장하는 혹은 제거하는 것이 좋습니다. 그런데 약 2년 동안 6.5mm 커진 ...
4,처녀막 재생수술 궁금증 안녕하세요. 제가 요즘 고민이 하나 생겼는데요. 아직 성...,"처녀막의 검사는 질입구에서 확인이 되므로 특별한 기구를 사용치 않으며, 간단합니다..."


## 2.2. Embedding

In [9]:
from sentence_transformers import SentenceTransformer

# https://huggingface.co/thenlper/gte-large
embedding_model = SentenceTransformer("thenlper/gte-large")

def get_embedding(text: str) -> list[float]:
    if not text.strip():
      print("empty")
      return []
    return embedding_model.encode(text).tolist()

dataset_df["embedding"] = dataset_df["instruction"].apply(get_embedding)
dataset_df.head(5)

,instruction,output,embedding
0,벌어진 앞니 레진 떼우기 이정도 벌어진 앞니 음식 씹을 때마다 너무 불편해레진 떼우...,가능은 하지만 일단 색이 치아 색과 맞지 않습니다 그리고 잘 떨어진다는 단점이 있습...,"[0.015179361216723919, 0.012804226949810982, 0..."
1,"유전형 탈모 검사법이 있는지 궁금합니다. 저희집안은 친가, 외가를 통틀어서 탈모가 ...",유전성 남성형 탈모에 있어서 탈모유전자가 하나는아니며유전자 검사가 있지만 그검사는...,"[0.017245162278413773, -0.0031242817640304565,..."
2,고추에 때? 흰색 물질이 있어요(급) 고추를 까보면 귀두에 흰색이 있는데요 일반 오...,음경부분에 흰색물질이 발견되어 고민이시군요.육안으로 확인되지 않아 판단이 어려우나 ...,"[-0.010009639896452427, -0.004838632885366678,..."
3,17년12월 유방섬유선종 1.15cm 유방외과 선생님 좀 봐주세요.17년12월 유방...,일단 성장하는 혹은 제거하는 것이 좋습니다. 그런데 약 2년 동안 6.5mm 커진 ...,"[0.017844077199697495, 0.005956630222499371, 0..."
4,처녀막 재생수술 궁금증 안녕하세요. 제가 요즘 고민이 하나 생겼는데요. 아직 성...,"처녀막의 검사는 질입구에서 확인이 되므로 특별한 기구를 사용치 않으며, 간단합니다...","[0.016576971858739853, 0.004225320648401976, 0..."


# 3. mongodb

mongodb Network Access

In [10]:
!curl ipecho.net/plain

34.124.176.98

connect to mongodb

In [11]:
import pymongo
from google.colab import userdata

def get_mongo_client(mongo_uri):
    """Establish connection to the MongoDB."""
    try:
        client = pymongo.MongoClient(mongo_uri)
        print("Connection to MongoDB successful")
        return client
    except pymongo.errors.ConnectionFailure as e:
        print(f"Connection failed: {e}")
        return None

mongo_uri = userdata.get("MONGO_URI")
if not mongo_uri:
    print("MONGO_URI not set in environment variables")

mongo_client = get_mongo_client(mongo_uri)

# Ingest data into MongoDB
db = mongo_client["spider504"]
collection = db["240703_movie_collection"]

Connection to MongoDB successful


In [26]:
collection.delete_many({})

documents = dataset_df.to_dict("records")
collection.insert_many(documents)

print("Data ingestion into MongoDB completed")

Data ingestion into MongoDB completed


# 4. Vector Search

In [12]:
def vector_search(user_query, collection):
    """
    Perform a vector search in the MongoDB collection based on the user query.

    Args:
    user_query (str): The user's query string.
    collection (MongoCollection): The MongoDB collection to search.

    Returns:
    list: A list of matching documents.
    """

    # Generate embedding for the user query
    query_embedding = get_embedding(user_query)

    if query_embedding is None:
        return "Invalid query or embedding generation failed."

    # Define the vector search pipeline
    pipeline = [
        {
            "$vectorSearch": {
                "index": "vector_index",
                "queryVector": query_embedding,
                "path": "embedding",
                "numCandidates": 150,  # Number of candidate matches to consider
                "limit": 4,  # Return top 4 matches
            }
        },
        {
            "$project": {
                "_id": 0,  # Exclude the _id field
                "instruction": 1,
                "output": 1,
                "score": {"$meta": "vectorSearchScore"},  # Include the search score
            }
        },
    ]

    results = collection.aggregate(pipeline)

    return list(results)

In [13]:
def get_search_result(query, collection):

    get_knowledge = vector_search(query, collection)

    search_result = ""
    for result in get_knowledge:
        search_result += f"instruction: {result.get('instruction', 'N/A')}\n"
        search_result += f"output: {result.get('output', 'N/A')}\n"
        search_result += f"score: {result.get('score', 'N/A')}\n"
    return search_result

# 5. Prompt

In [14]:
# Conduct query with retrival of sources
query = "왼쪽 무릎관절에 통증이 있어요."
source_information = get_search_result(query, collection)

combined_information = (
f"""
Query: {query}
Continue to answer the query by using the Search Results.
If you need more information, search information through below <documents> tag.
You should answer with Korean.
<documents>
{source_information}
"""
)

print(combined_information)


Query: 왼쪽 무릎관절에 통증이 있어요.
Continue to answer the query by using the Search Results.
If you need more information, search information through below <documents> tag.
You should answer with Korean.
<documents>
instruction: 티눈?사마귀?아파죽겟어요ㅜㅜ 아너무아파요ㅠ티눈이랑사마군같이없애는 그린티눈액사서했는데 아진ㅉㅏ아파서손도못대겠어요ㅠ티눈인줄알앗는데어떤사람은사마귀라고도하고ㅠ티눈인가요사마괴인가요ㅠ발뒤꿈치에나서걷기도불편하고ㅠ 피부과가야하나요
output: 직접 진료와 같을 수 없기에 첨부한 사진과 서면상의 증상만으로티눈,사마귀를 구분하여 말씀 드리기에는 어려움이 있습니다.특히 티눈액을 발라서 피부 병변의 모양이 바뀌어 정확히 볼 수가 없군요,아마도 사마귀 가능성이 더 높을 것 같습니다.흔히 티눈이라고 생각하는 병변 중에 사마귀인 경우가 상당히 많습니다. 티눈은 지속적인 압박에 의해 생기며각질을 깎아냈을 때 검은 점 모양의 점상출혈을 보이지 않고중심핵이 관찰되며 위에서 눌렀을 때 통증이 느껴집니다.사마귀는 바이러스에 의한 감염성 질환으로 크기가 커지거나 다른 부위로 번질 수도 있습니다.발에 생기는 사마귀나 티눈은 깊이가 깊어서 티눈밴드만으로는 완치를 기대하기가어렵습니다.통증 증상이 나타나고 있으므로가까운 피부과에서치료하시는 게 좋을 듯 합니다.감사합니다
score: 0.9624912738800049
instruction: 몽정안했는데액이.. 제고밍좀해결해주세요..야동을보고있었습니다.자위를하고있었구여..12시33분이에여근데정액이나왔어요..하얀액체가..그런데정액에는아무런색깔없이오직하얀색..피도안섞여있었규..왜그럴까요?몽정도안했는디....이런고민햐결좀요..
output: 너무 과도하게 흥분이 된 경우 가능한 일입니다.전혀 걱정할 일이 아니지만 같은 증상이 계속 될 경우에는진료를 받아보시길 바라겠습니다.답변

# 6. loading gemma

## 6.1. QLoRA

In [15]:
import torch
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [16]:
from transformers import AutoTokenizer, AutoModelForCausalLM

BASE_MODEL = "google/gemma-2b-it"

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [17]:
raw_input_ids = tokenizer(query, return_tensors="pt").to("cuda")
raw_response = model.generate(**raw_input_ids, max_new_tokens=500)
print('### only Gemma')
print(tokenizer.decode(raw_response[0]))

input_ids = tokenizer(combined_information, return_tensors="pt").to("cuda")
response = model.generate(**input_ids, max_new_tokens=500)
print('### Gemma on RAG')
print(tokenizer.decode(response[0]))

### only Gemma
<bos>왼쪽 무릎관절에 통증이 있어요.

왼쪽 무릎관절 통증의 주요 증상은 무릎의 굴곡이 어지거나 굽혀지지 않다는 것입니다. 또한, 통증은 무릎관절의 움직임이 어려워 움직이지 않는다는 것입니다.

왼쪽 무릎관절 통증은 어떻게 발생했나요?

**답:**

* 왼쪽 무릎관절 통증은 굴곡이 어지거나 굽혀지지 않다는 증상으로 발생할 수 있습니다.
* 왼쪽 무릎관절 통증은 움직임이 어려워 움직이지 않는다는 증상으로 발생할 수 있습니다.<eos>
### Gemma on RAG
<bos>
Query: 왼쪽 무릎관절에 통증이 있어요.
Continue to answer the query by using the Search Results.
If you need more information, search information through below <documents> tag.
You should answer with Korean.
<documents>
instruction: 티눈?사마귀?아파죽겟어요ㅜㅜ 아너무아파요ㅠ티눈이랑사마군같이없애는 그린티눈액사서했는데 아진ㅉㅏ아파서손도못대겠어요ㅠ티눈인줄알앗는데어떤사람은사마귀라고도하고ㅠ티눈인가요사마괴인가요ㅠ발뒤꿈치에나서걷기도불편하고ㅠ 피부과가야하나요
output: 직접 진료와 같을 수 없기에 첨부한 사진과 서면상의 증상만으로티눈,사마귀를 구분하여 말씀 드리기에는 어려움이 있습니다.특히 티눈액을 발라서 피부 병변의 모양이 바뀌어 정확히 볼 수가 없군요,아마도 사마귀 가능성이 더 높을 것 같습니다.흔히 티눈이라고 생각하는 병변 중에 사마귀인 경우가 상당히 많습니다. 티눈은 지속적인 압박에 의해 생기며각질을 깎아냈을 때 검은 점 모양의 점상출혈을 보이지 않고중심핵이 관찰되며 위에서 눌렀을 때 통증이 느껴집니다.사마귀는 바이러스에 의한 감염성 질환으로 크기가 커지거나 다른 부위로 번질 수도 있습니다.발에 생기는 사마귀나 티눈은 깊이가 깊어서 티눈밴드만으로는 완치를 기대하기가어렵습니다.통증 증상이 나타나고 있으므로가까